In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import csv
from urllib.parse import urljoin
from time import sleep

### 最新記事データの取得

In [2]:
url = "https://gihyo.jp"
article_list = []

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

for elem in soup.select("div#latestArticle h3 a"):
    url = urljoin(url, elem["href"])
    
    article_url = url
    article_text = ""
    for i in range(9):
    # while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        article_title = soup.select("div#primary h2")[0].text.replace("\u3000","").replace("\n","")
        publish_date = soup.select("p.date")[0].text
        article_text = article_text + soup.select("div.readingContent01")[0].text.replace("\u3000","").replace("\u2060","").replace("\n","").replace("\r","")
        sleep(1)
        
        if soup.select("a[rel='next']"):
            url = urljoin(url, soup.select("a[rel='next']")[0]["href"])
        else:
            article_list.append([article_title, article_url, publish_date, article_text])
            break     

### 参考： 取得したデータの表示

In [3]:
for article in article_list:
    print(article, end="\n\n")

['2021年9月第5週号 1位は，UXデザインのグローバル化についてのヒント，気になるネタは，米Slackが動画を簡単に作成・共有できる新機能などを発表', 'https://gihyo.jp/design/clip/01/weekly-web-tech/202109/30', '2021年9月30日', 'ネットで見かけたWebテク（Webテクニック・Webテクノロジー）記事から，Webデザイナーの目で厳選したネタを週刊で紹介するこのコーナー。今回は，2021年9月20日～9月26日の間に見つけた記事のベスト5です。1. Globalizing Your UX Designs | by Nicole Assini | Indeed Designhttps://indeed.design/article/globalizing-your-ux-designsUXデザインのグローバル化についてのヒントを紹介した記事です。英語をベースにローカライズするときに，ただ他の言語に翻訳するだけなく考慮すべき点をまとめています。文字数が多くなることを計画しておく（英語から他言語に翻訳するとテキスト量が長くなることが多い）言語によって文字サイズが変動することを考慮する単純明快な言葉を使う入力フィールドは調整可能にする（日本では姓・名の順番が米国と逆でふりがなも必要など）国によって固有のフィールドの並び順がある（住所の表記が米国では番地→市区町村→州なのに対し日本は逆など）通貨記号とその配置は国や言語によって違うコンテンツとインタラクションを柔軟に情報の階層性を保つ語順が変更されることを計画しておく（文章中の穴埋めのようなフォームを作らないなど）右から左に書く言語のためにミラーリングすることを考慮する画像とシンボルは誤解される可能性がある色が持つ意味合いは国によって違う場合があるテキスト入りの画像を作らない図1UXデザインのグローバル化についてのヒント2. The Pros and Cons of Tailwind CSS | Webdesigner Depot Webdesigner Depot » Blog Archivehttps://www.webdesignerdepot.com/2021/09/the-pros-and-cons-of-tailwind-css/Tail

### 取得した記事データの保存

In [4]:
csv_header = ["タイトル","URL","発行日","本文"]
csv_date = datetime.datetime.today().strftime("%Y%m%d%H")
csv_file_name = "gihyo_" + csv_date + ".csv"
with open(csv_file_name,"w", errors = "ignore") as file:
    writer = csv.writer(file,lineterminator="\n")
    writer.writerow(csv_header)
    writer.writerows(article_list)